In [1]:
import urllib.request as urllib
import zipfile
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import matplotlib.pyplot as plt
import time
import numpy as np
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", 
          "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", 
          "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [87]:
url = 'https://gaftp.epa.gov/air/nei/2017/data_summaries/2017v1/2017neiJan_county_tribe_allsector.zip'
filehandle, _ = urllib.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
my_file = zip_file_object.namelist()[1]
df = pd.read_csv(zip_file_object.open(my_file))
zip_file_object.close()

In [88]:
df = df.filter(['state', 'fips code', 'county', 'sector', 'pollutant desc','pollutant type(s)', 'total emissions', 'emissions uom'])

In [89]:
df['pollutant type(s)'].unique()

array(['HAP', 'CAP', nan, 'CAP/HAP', 'GHG'], dtype=object)

In [139]:
dfghg = df[(df['pollutant type(s)']=='GHG')&(~dfcap.state.isnull())].reset_index(drop=True)
dfcap = df[(df['pollutant type(s)']=='CAP')&~dfcap.state.isnull()].reset_index(drop=True)
dfcap = dfcap[(dfcap['pollutant desc']!='PM10 Filterable')&(dfcap['pollutant desc']!='PM Condensible')&(dfcap['pollutant desc']!='PM2.5 Filterable')]

In [140]:
dfghg.to_csv('2017_EPA_NEI_GHG.csv')
dfcap.to_csv('2017_EPA_NEI_CAP.csv')

In [145]:
def get_county_cap_data(sa):
    caps = pd.DataFrame(columns=['State','County','Sector','Pollutant','Pollutant Type','Total Emissions','Emissions Unit of Measure'])
    df2 = dfcap[dfcap['state']==sa]
    for c in df2['county'].unique():
        for p in df2['pollutant desc'].unique():
            for s in df2['sector'].unique():
                try:
                    dft2 = df2[(df2['pollutant desc']==p)&(df2['sector']==s)].reset_index(drop=True)
                    caps.loc[len(caps.index)] = [sa, c, s, p, 'CAP', sum(dft2['total emissions']), dft2['emissions uom'][0]]
                except:
                    None
            dft2 = df2[(df2['pollutant desc']==p)].reset_index(drop=True)
            caps.loc[len(caps.index)] = [sa, c, 'All Sectors', p, 'CAP', sum(dft2['total emissions']), dft2['emissions uom'][0]]
    caps = caps.sort_values(by=['County','Sector','Pollutant']).reset_index(drop=True)
    return caps

def get_state_cap_data(sa):
    caps = pd.DataFrame(columns=['State','Sector','Pollutant','Pollutant Type','Total Emissions','Emissions Unit of Measure'])
    df2 = dfcap[dfcap['state']==sa]
    for p in df2['pollutant desc'].unique():
        for s in df2['sector'].unique():
            try:
                dft2 = df2[(df2['pollutant desc']==p)&(df2['sector']==s)].reset_index(drop=True)
                caps.loc[len(caps.index)] = [sa, s, p, 'CAP', sum(dft2['total emissions']), dft2['emissions uom'][0]]
            except:
                None
        dft2 = df2[(df2['pollutant desc']==p)].reset_index(drop=True)
        caps.loc[len(caps.index)] = [sa, 'All Sectors', p, 'CAP', sum(dft2['total emissions']), dft2['emissions uom'][0]]
    caps = caps.sort_values(by=['Sector', 'Pollutant']).reset_index(drop=True)
    return caps

def get_national_cap_data():
    caps = pd.DataFrame(columns=['Sector','Pollutant','Pollutant Type','Total Emissions','Emissions Unit of Measure'])
    df2 = dfcap[~dfcap.state.isnull()].state.unique()
    for p in df2['pollutant desc'].unique():
        for s in df2['sector'].unique():
            try:
                dft2 = df2[(df2['pollutant desc']==p)&(df2['sector']==s)].reset_index(drop=True)
                caps.loc[len(caps.index)] = [s, p,'CAP',  sum(dft2['total emissions']), dft2['emissions uom'][0]]
            except:
                None
        dft2 = df2[(df2['pollutant desc']==p)].reset_index(drop=True)
        caps.loc[len(caps.index)] = ['All Sectors', p,'CAP',  sum(dft2['total emissions']), dft2['emissions uom'][0]]
    caps = caps.sort_values(by=['Sector', 'Pollutant']).reset_index(drop=True)
    return caps

In [146]:
def get_county_ghg_data(sa):
    ghgs = pd.DataFrame(columns=['State','County','Sector','Pollutant','Pollutant Type','Total Emissions','Emissions Unit of Measure'])
    df2 = dfghg[dfghg['state']==sa]
    for c in df2['county'].unique():
        for p in df2['pollutant desc'].unique():
            for s in df2['sector'].unique():
                try:
                    dft2 = df2[(df2['pollutant desc']==p)&(df2['sector']==s)].reset_index(drop=True)
                    ghgs.loc[len(ghgs.index)] = [sa, c, s, p,'GHG',  sum(dft2['total emissions']), dft2['emissions uom'][0]]
                except:
                    None
            dft2 = df2[(df2['pollutant desc']==p)].reset_index(drop=True)
            ghgs.loc[len(ghgs.index)] = [sa, c, 'All Sectors', p,'GHG',  sum(dft2['total emissions']), dft2['emissions uom'][0]]
    ghgs = ghgs.sort_values(by=['County','Sector','Pollutant']).reset_index(drop=True)
    return ghgs

def get_state_ghg_data(sa):
    ghgs = pd.DataFrame(columns=['State','Sector','Pollutant','Pollutant Type','Total Emissions','Emissions Unit of Measure'])
    df2 = dfghg[dfghg['state']==sa]
    for p in df2['pollutant desc'].unique():
        for s in df2['sector'].unique():
            try:
                dft2 = df2[(df2['pollutant desc']==p)&(df2['sector']==s)].reset_index(drop=True)
                ghgs.loc[len(ghgs.index)] = [sa, s, p, 'GHG', sum(dft2['total emissions']), dft2['emissions uom'][0]]
            except:
                None
        dft2 = df2[(df2['pollutant desc']==p)].reset_index(drop=True)
        ghgs.loc[len(ghgs.index)] = [sa, 'All Sectors', p,'GHG',  sum(dft2['total emissions']), dft2['emissions uom'][0]]
    ghgs = ghgs.sort_values(by=['Sector','Pollutant']).reset_index(drop=True)
    return ghgs

def get_national_ghg_data():
    ghgs = pd.DataFrame(columns=['Sector','Pollutant','Pollutant Type','Total Emissions','Emissions Unit of Measure'])
    df2 = dfghg[~dfghg.state.isnull()].state.unique()
    for p in df2['pollutant desc'].unique():
        for s in df2['sector'].unique():
            try:
                dft2 = df2[(df2['pollutant desc']==p)&(df2['sector']==s)].reset_index(drop=True)
                ghgs.loc[len(ghgs.index)] = [s, p, 'GHG', sum(dft2['total emissions']), dft2['emissions uom'][0]]
            except:
                None
        dft2 = df2[(df2['pollutant desc']==p)].reset_index(drop=True)
        ghgs.loc[len(ghgs.index)] = ['All Sectors', p, 'GHG', sum(dft2['total emissions']), dft2['emissions uom'][0]]
    ghgs = ghgs.sort_values(by=['Sector', 'Pollutant']).reset_index(drop=True)
    return ghgs

In [158]:
def get_county_emissions(sa):
    caps= get_county_cap_data(sa)
    ghgs = get_county_ghg_data(sa)
    dfnew = pd.concat([caps, ghgs], axis=0).sort_values(by=['County','Sector','Pollutant Type','Pollutant'])
    dftots = dfnew[dfnew['Sector']=='All Sectors']
    dfnontots = dfnew[dfnew['Sector']!='All Sectors']
    dfnew = pd.concat([dftots, dfnontots], axis=0)
    return dfnew

def get_state_emissions(sa):
    caps= get_state_cap_data(sa)
    ghgs = get_state_ghg_data(sa)
    dfnew = pd.concat([caps, ghgs], axis=0).sort_values(by=['Sector','Pollutant Type','Pollutant'])
    dftots = dfnew[dfnew['Sector']=='All Sectors']
    dfnontots = dfnew[dfnew['Sector']!='All Sectors']
    dfnew = pd.concat([dftots, dfnontots], axis=0)
    return dfnew

def get_national_emissions(sa):
    caps= get_national_cap_data()
    ghgs = get_national_ghg_data()
    dfnew = pd.concat([caps, ghgs], axis=0).sort_values(by=['Sector','Pollutant Type','Pollutant'])
    dftots = dfnew[dfnew['Sector']=='All Sectors']
    dfnontots = dfnew[dfnew['Sector']!='All Sectors']
    dfnew = pd.concat([dftots, dfnontots], axis=0)
    return dfnew

In [147]:
caps= get_state_cap_data('WY')
ghgs = get_state_ghg_data('WY')

In [156]:
dfnew = pd.concat([caps, ghgs], axis=0).sort_values(by=['Sector','Pollutant Type','Pollutant'])
dftots = dfnew[dfnew['Sector']=='All Sectors']
dfnontots = dfnew[dfnew['Sector']!='All Sectors']
dfnew = pd.concat([dftots, dfnontots], axis=0)

In [157]:
dfnew.head(15)

,State,Sector,Pollutant,Pollutant Type,Total Emissions,Emissions Unit of Measure
4,WY,All Sectors,Ammonia,CAP,885.812912,TON
5,WY,All Sectors,Carbon Monoxide,CAP,10101.106682,TON
6,WY,All Sectors,Nitrogen Oxides,CAP,6182.972001,TON
7,WY,All Sectors,PM10 Primary (Filt + Cond),CAP,3988.369464,TON
8,WY,All Sectors,PM2.5 Primary (Filt + Cond),CAP,498.288649,TON
9,WY,All Sectors,Sulfur Dioxide,CAP,467.948089,TON
10,WY,All Sectors,Volatile Organic Compounds,CAP,13620.810174,TON
0,WY,All Sectors,Carbon Dioxide,GHG,692243.046126,TON
1,WY,All Sectors,Methane,GHG,751.356290,TON
2,WY,All Sectors,Nitrous Oxide,GHG,0.228438,TON
